In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import chain


In [2]:
#Bringing in data
denver_listing = "../raw_data/listings_denver.csv"
raw_data = pd.read_csv(denver_listing)
raw_data.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,177,https://www.airbnb.com/rooms/177,2.021070e+13,7/23/2021,Tiny Home in the Heart of the City- ECO FRIENDLY,"160 sq ft + 80 sq ft loft for sleeping, Sleeps...","Quiet neighborhood next to park, creeks and bi...",https://a0.muscache.com/pictures/a1745ab2-b3a0...,615,https://www.airbnb.com/users/show/615,...,4.93,4.83,4.76,2016-BFN-0004968,f,2,2,0,0,1.49
1,360,https://www.airbnb.com/rooms/360,2.021070e+13,7/23/2021,Sit in the Peaceful Garden of the Chickadee Co...,Enjoy the famous Colorado weather and unplug i...,The cottage is located in the center of Lower ...,https://a0.muscache.com/pictures/monet/Select-...,666,https://www.airbnb.com/users/show/666,...,5.00,5.00,4.91,2017-BFN-0002177,f,2,2,0,0,8.98
2,364,https://www.airbnb.com/rooms/364,2.021070e+13,7/23/2021,Lodo / RiNo LOFT via airport train,"Modern 1,000 square foot loft in the heart of ...","Ten brewpubs within walking distance, two grea...",https://a0.muscache.com/pictures/11766413/a2c5...,783,https://www.airbnb.com/users/show/783,...,4.96,4.65,4.71,NaN,f,1,1,0,0,0.70


In [3]:
#Looking to see what columns would be helpful for comparision to a new unlisted property
raw_data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [4]:
#Exploring the data
#I will need to extract the array from the amenities to be be able to use that information.
data = raw_data[['price','property_type','bedrooms','bathrooms_text', 'beds', 'neighbourhood_cleansed','amenities']]
data.head()

,price,property_type,bedrooms,bathrooms_text,beds,neighbourhood_cleansed,amenities
0,$99.00,Entire guesthouse,NaN,1 bath,0.0,Washington Virginia Vale,"[""Patio or balcony"", ""Shampoo"", ""BBQ grill"", ""..."
1,$135.00,Entire guesthouse,2.0,1 bath,2.0,Highland,"[""TV"", ""Fire extinguisher"", ""Heating"", ""Wifi"",..."
2,$179.00,Entire loft,1.0,1.5 baths,1.0,Five Points,"[""Shampoo"", ""Air conditioning"", ""Carbon monoxi..."
3,$55.00,Private room in residential home,1.0,1 shared bath,1.0,North Park Hill,"[""Oven"", ""Patio or balcony"", ""Shampoo"", ""Dishw..."
4,$52.00,Private room in residential home,1.0,1 shared bath,1.0,North Park Hill,"[""Oven"", ""Shampoo"", ""Dishwasher"", ""Air conditi..."


In [5]:
# I will have to price, rooms to numbers
print(data.dtypes)
print(data.shape)

price                      object
property_type              object
bedrooms                  float64
bathrooms_text             object
beds                      float64
neighbourhood_cleansed     object
amenities                  object
dtype: object
(3697, 7)


In [6]:
#cleaning up price column
data['price'] = data['price'].str.replace('$', '')

<ipython-input-6-1e4935a145ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['price'].str.replace('$', '')


In [7]:
data['price'] = pd.to_numeric(data['price'], errors='coerce')

<ipython-input-7-ecd07f53d261>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = pd.to_numeric(data['price'], errors='coerce')


In [8]:
#Checking for nan values
data.isnull().sum()

price                      35
property_type               0
bedrooms                  222
bathrooms_text              3
beds                      135
neighbourhood_cleansed      0
amenities                   0
dtype: int64

In [9]:
# clear nan values
data = data.dropna()

In [10]:
print(data.isnull().sum())
print(data.shape)

price                     0
property_type             0
bedrooms                  0
bathrooms_text            0
beds                      0
neighbourhood_cleansed    0
amenities                 0
dtype: int64
(3307, 7)


In [11]:
data.dtypes

price                     float64
property_type              object
bedrooms                  float64
bathrooms_text             object
beds                      float64
neighbourhood_cleansed     object
amenities                  object
dtype: object

In [12]:
#time to analyse the amenities column closer
data['amenities'].head()

1    ["TV", "Fire extinguisher", "Heating", "Wifi",...
2    ["Shampoo", "Air conditioning", "Carbon monoxi...
3    ["Oven", "Patio or balcony", "Shampoo", "Dishw...
4    ["Oven", "Shampoo", "Dishwasher", "Air conditi...
5    ["Extra pillows and blankets", "Oven", "Patio ...
Name: amenities, dtype: object

In [13]:
#the row is not view as a list in this data frame
data['amenities'].value_counts()

["Oven", "Shampoo", "Dishwasher", "Air conditioning", "Dishes and silverware", "Dryer", "Bed linens", "Smoke alarm", "TV", "Essentials", "Hangers", "Kitchen", "Coffee maker", "Dedicated workspace", "Iron", "Refrigerator", "Hair dryer", "Elevator", "Microwave", "Cooking basics", "Stove", "Lockbox", "Heating", "Wifi", "Long term stays allowed", "Washer"]                                                                                                                                                                                                                                                                                                                                                                                                                                                    22
["Hot water", "Iron", "Elevator", "Coffee maker", "Central heating", "Paid valet parking on premises", "Single level home", "Dryer", "Hair dryer", "Refrigerator", "Oven", "Microwave", "Shampoo", "Security ca

In [14]:
data['amenities'] = data['amenities'].apply(eval)

In [15]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

In [16]:
#now a list can be extracted to create multiple columns for later comparison
to_1D(data['amenities']).value_counts()

Smoke alarm                                               3257
Essentials                                                3182
Wifi                                                      3165
Carbon monoxide alarm                                     3151
Hangers                                                   3010
                                                          ... 
Bosch Stainless Steel Single Oven stainless steel oven       1
generic conditioner                                          1
Olay, Suave, Tresemme body soap                              1
Electric burner stove                                        1
GE Gas Range oven                                            1
Length: 882, dtype: int64

In [17]:
a1 = to_1D(data['amenities']).unique().tolist()

In [18]:
#I have a sorted list of all amminetes now that I can generate individual columns have a lot of repeated and similar values. 
print(len(a1))

882


In [19]:
# Now I have a list of all avalible amenities
a1[:10]

['TV',
 'Fire extinguisher',
 'Heating',
 'Wifi',
 'Smoke alarm',
 'Shared hot tub',
 'Iron',
 'Free street parking',
 'Coffee maker',
 'Full kitchen']

In [20]:
data['amenities'].dtypes

dtype('O')

In [21]:
amenities = pd.DataFrame(data['amenities'].tolist())

In [22]:
amenities.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
0,TV,Fire extinguisher,Heating,Wifi,Smoke alarm,Shared hot tub,Iron,Free street parking,Coffee maker,Full kitchen,...,None,None,None,None,None,None,None,None,None,None
1,Shampoo,Air conditioning,Carbon monoxide alarm,Dryer,Smoke alarm,Essentials,Hangers,Kitchen,Free parking on premises,Breakfast,...,None,None,None,None,None,None,None,None,None,None
2,Oven,Patio or balcony,Shampoo,Dishwasher,Carbon monoxide alarm,BBQ grill,Dishes and silverware,Dryer,Hot water,Smoke alarm,...,None,None,None,None,None,None,None,None,None,None
3,Oven,Shampoo,Dishwasher,Air conditioning,Carbon monoxide alarm,Dishes and silverware,Dryer,Hot water,Lock on bedroom door,Smoke alarm,...,None,None,None,None,None,None,None,None,None,None
4,Extra pillows and blankets,Oven,Patio or balcony,Shampoo,Dishwasher,Air conditioning,Carbon monoxide alarm,Dishes and silverware,Dryer,Hot water,...,None,None,None,None,None,None,None,None,None,None


In [23]:
amenities.index = amenities.index + 1

In [24]:
amenities.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
1,TV,Fire extinguisher,Heating,Wifi,Smoke alarm,Shared hot tub,Iron,Free street parking,Coffee maker,Full kitchen,...,None,None,None,None,None,None,None,None,None,None
2,Shampoo,Air conditioning,Carbon monoxide alarm,Dryer,Smoke alarm,Essentials,Hangers,Kitchen,Free parking on premises,Breakfast,...,None,None,None,None,None,None,None,None,None,None
3,Oven,Patio or balcony,Shampoo,Dishwasher,Carbon monoxide alarm,BBQ grill,Dishes and silverware,Dryer,Hot water,Smoke alarm,...,None,None,None,None,None,None,None,None,None,None
4,Oven,Shampoo,Dishwasher,Air conditioning,Carbon monoxide alarm,Dishes and silverware,Dryer,Hot water,Lock on bedroom door,Smoke alarm,...,None,None,None,None,None,None,None,None,None,None
5,Extra pillows and blankets,Oven,Patio or balcony,Shampoo,Dishwasher,Air conditioning,Carbon monoxide alarm,Dishes and silverware,Dryer,Hot water,...,None,None,None,None,None,None,None,None,None,None


In [25]:
#Exploring the neighourhood vs price
data['neighbourhood_cleansed'].unique()

array(['Highland', 'Five Points', 'North Park Hill', 'North Capitol Hill',
       'West Highland', 'Capitol Hill', 'Overland', 'South Park Hill',
       'CBD', 'Washington Park', 'Union Station', 'Washington Park West',
       'Baker', 'City Park West', 'Clayton', 'Stapleton', 'Platt Park',
       'Lowry Field', 'Sunnyside', 'Congress Park', 'Goldsmith',
       'Berkeley', 'Lincoln Park', 'Cole', 'Cheesman Park', 'Speer',
       'University', 'West Colfax', 'Sloan Lake', 'Hilltop',
       'Country Club', 'Cory - Merrill', 'City Park', 'Athmar Park',
       'Washington Virginia Vale', 'Gateway - Green Valley Ranch',
       'Rosedale', 'Whittier', 'Fort Logan', 'Skyland', 'Windsor', 'Hale',
       'Cherry Creek', 'Harvey Park', 'Ruby Hill', 'Hampden South',
       'Civic Center', 'Jefferson Park', 'Montclair', 'Harvey Park South',
       'Regis', 'University Park', 'Hampden', 'Barnum West',
       'Virginia Village', 'Barnum', 'University Hills',
       'Northeast Park Hill', 'East Colfa

In [26]:
area_price = data.groupby('neighbourhood_cleansed', as_index=False)['price'].mean()

In [27]:
 area_price.sort_values(by='price') 

,neighbourhood_cleansed,price
44,Montbello,68.909091
34,Harvey Park South,73.000000
39,Kennedy,78.333333
18,College View - South Platte,79.000000
26,Fort Logan,82.500000
...,...,...
72,West Colfax,231.428571
6,Belcaro,232.250000
21,Country Club,267.454545
15,Civic Center,311.600000


In [28]:
#Giving a numerical value to shared or no shared bathroom
data['bathrooms_text'].unique()

array(['1 bath', '1.5 baths', '1 shared bath', '1 private bath',
       '4 baths', '2.5 baths', '1.5 shared baths', '2 baths', '3.5 baths',
       '3 baths', '4.5 baths', 'Shared half-bath', '5 baths',
       '2 shared baths', '17 baths', '2.5 shared baths', '4 shared baths',
       '0 baths', '0 shared baths'], dtype=object)

In [29]:
# Removing non numerical charcaters from bathrooms_text and replacing the empty values
data['bathrooms_text'] = data['bathrooms_text'].map(lambda x: ''.join([i for i in x if i.isdigit() or i =='.']))

In [30]:
data['bathrooms_text'] = data['bathrooms_text'].replace('', 0)

In [31]:
data['bathrooms_text'].unique()

array(['1', '1.5', '4', '2.5', '2', '3.5', '3', '4.5', 0, '5', '17', '0'],
      dtype=object)

In [32]:
#Converting column type into float type to retain half bathrooms
data['bathrooms_text'] = data['bathrooms_text'].astype('float')

In [33]:
data.dtypes

price                     float64
property_type              object
bedrooms                  float64
bathrooms_text            float64
beds                      float64
neighbourhood_cleansed     object
amenities                  object
dtype: object

In [34]:
# Droping ammenites for now as I'm sorta lost on how to apply this to the df at this time
clean_data = data.drop(['amenities'], axis=1)

In [35]:
clean_data.shape

(3307, 6)

In [40]:
# Saving cleaned data.
clean_data.to_csv(r'..\data\clean_data.csv', index = True)

In [41]:
# Save amenities separately for now.
clean_data.to_csv(r'..\data\amenites.csv', index = True)

In [43]:
df_neigh = pd.get_dummies(clean_data)

In [46]:
df_neigh.shape

(3307, 119)